# Fastplotlib Examples

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import tifffile
import fastplotlib as fpl
import matplotlib.pyplot as plt

from skophys.preprocessing import Vectorizer, UnVectorizer, PercentileFilter

import lbm_caiman_python as lcp

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Available devices:
✅ (default) | NVIDIA RTX A4000 | DiscreteGPU | Vulkan | 560.94
✅ | NVIDIA RTX A4000 | DiscreteGPU | D3D12 | 
❗ | Microsoft Basic Render Driver | CPU | D3D12 | 
✅ | NVIDIA RTX A4000 | DiscreteGPU | D3D12 | 
❗ | NVIDIA RTX A4000/PCIe/SSE2 | Unknown | OpenGL | 


c:\Users\RBO\miniforge3\envs\lcp\lib\site-packages\numba\core\decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [6]:
movie_path = Path().home() / 'lbm_data' / 'assembled' / 'roi_0_plane_24.tiff'

In [7]:
movie = tifffile.imread(movie_path)
movie.shape

(1000, 212, 212)

## Smoothing with Gaussian Filter

In [ ]:
## slider
from ipywidgets import IntSlider, VBox
slider_gsig_filt = IntSlider(value=3, min=1, max=33, step=1,  description="gSig_filt")

In [9]:
from caiman.motion_correction import high_pass_filter_space

def apply_filter(frame):
    gSig_filt = (slider_gsig_filt.value, slider_gsig_filt.value)

    # apply filter
    return high_pass_filter_space(frame, gSig_filt)

# filter shown on 2 right plots, index 1 and 2
funcs = {1:apply_filter}

In [ ]:
iw = fpl.ImageWidget(
    data=[movie[:500], movie[:500]], # we'll apply the filter to the second movie
    frame_apply=funcs,
    figure_kwargs={"size": (1200, 600)},
    names=['raw', 'filtered'],
    cmap="gnuplot2"
)
iw.figure[0, 0].auto_scale()
iw.figure[0, 1].auto_scale()

iw.figure["filtered"].set_title(f"filtered: σ={slider_gsig_filt.value}")
iw.window_funcs = {"t": (np.mean, 3)}

def force_update(*args):
    # forces the images to update when the gSig_filt slider is moved
    iw.current_index = iw.current_index
    iw.reset_vmin_vmax()
    iw.figure["filtered"].set_title(f"filtered: σ={slider_gsig_filt.value}")

iw.reset_vmin_vmax()

slider_gsig_filt.observe(force_update, "value")

VBox([iw.show(), slider_gsig_filt])

In [ ]:
iw.close()

## Preview 'raw traces'

Show the raw trace for a given pixel by clicking on that pixel.

In [12]:
iw_movie = fpl.ImageWidget(movie, cmap="viridis")

tfig = fpl.Figure()

raw_trace = tfig[0, 0].add_line(np.zeros(movie.shape[0]))

@iw_movie.managed_graphics[0].add_event_handler("click")
def pixel_clicked(ev):
    col, row = ev.pick_info["index"]
    raw_trace.data[:, 1] =  iw_movie.data[0][:, row, col]
    tfig[0, 0].auto_scale(maintain_aspect=False)

VBox([iw_movie.show(), tfig.show()])

RFBOutputContext()

RFBOutputContext()